In [1]:
import numpy
import numpy as np
import pandas as pd
import re
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# extract data from log file by the special keyword
def read_log(file, model_name):
    line_list = []
    with open(file) as f:
        while True:
            lines = f.readline()
            line_list.append(lines)
            if not lines:
                break
    # print(line_list)
    if model_name == 'Mlp':
        keyline = "INFO - Average testing results among all repeated 80-20 holdouts:\n"
    for i in range(len(line_list)):
        if line_list[i] == keyline:
            key_index = i
    srcc = line_list[key_index + 1]
    krcc = line_list[key_index + 2]
    plcc = line_list[key_index + 3]
    rmse = line_list[key_index + 4]
    return srcc, krcc, plcc, rmse

def get_value(coef, model_name):
    if model_name == 'Mlp':
        pattern = r"INFO - (\w+): (\d+\.\d+) \(std: (\d+\.\d+)\)"

    matches = re.findall(pattern, coef)
    for match in matches:
        metric, value, std_dev = match
        value = format(float(value), '.4f')
        std_dev = format(float(std_dev), '.4f')
        plusmius = u"\u00B1"
        coef_value = f"{value} ({plusmius}{std_dev})"
    return coef_value

def process_logs(data_name, feats_name, model_name, log_path, select_criteria):
    data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], [], []
    for data in data_name:
        for feat in feats_name:
            if data == 'lsvq_test':
                log_file = f'{log_path}/lsvq_train_{feat}_{model_name}_corr_{select_criteria}_kfold.log'
            elif data == 'lsvq_test_1080p':
                log_file = f'{log_path}/lsvq_train_{feat}_{model_name}_corr_{select_criteria}_kfold_lsvq1080p.log'
            else:
                log_file = f'{log_path}/{data}_{feat}_{model_name}_corr_{select_criteria}.log'
            if not os.path.exists(log_file):
                continue  # skip if the file doesn't exist
            with open(log_file, "r", encoding="UTF-8") as file:
                srcc, krcc, plcc, rmse = read_log(log_file, model_name)
                srcc = get_value(srcc, model_name)
                krcc = get_value(krcc, model_name)
                plcc = get_value(plcc, model_name)
                rmse = get_value(rmse, model_name)
    
            data_list.append(data)
            feats_list.append(feat)
    
            srcc_list.append(srcc)
            krcc_list.append(krcc)
            plcc_list.append(plcc)
            rmse_list.append(rmse)
    return data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list

def read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune):
    data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], [], []
    for feat in feats_name:
        for data in data_name:
            if is_finetune == True:
                log_file = f'{log_path}/{data}_{feat}_{select_criteria}_finetune.csv'
                ft_name = f'{feat}_finetune'
            else:
                log_file = f'{log_path}/{data}_{feat}_{select_criteria}_wo_finetune.csv'
                ft_name = f'{feat}_wo_finetune'
            # print(log_file)
            ft_df = pd.read_csv(log_file)
            selected_data = ft_df[ft_df['SELECT_CRITERIC'] == select_criteria]
            data_list.append(data)
            feats_list.append(ft_name)
            srcc_list.append(selected_data['SRCC'].iloc[0])
            krcc_list.append(selected_data['KRCC'].iloc[0])
            plcc_list.append(selected_data['PLCC'].iloc[0])
            rmse_list.append(selected_data['RMSE'].iloc[0])
    return data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list

def create_results_dataframe(data_list,  feats_list, srcc_list, krcc_list, plcc_list, rmse_list):
    df_results = pd.DataFrame(columns=['DATASET', 'FEATS_NAME', 'SRCC (±STD)', 'KRCC (±STD)', 'PLCC (±STD)', 'RMSE (±STD)'])

    df_results['DATASET'] = data_list
    df_results['FEATS_NAME'] =  feats_list
    df_results['SRCC (±STD)'] = srcc_list
    df_results['KRCC (±STD)'] = krcc_list
    df_results['PLCC (±STD)'] = plcc_list
    df_results['RMSE (±STD)'] = rmse_list
    return df_results

In [3]:
model_name = 'Mlp'
select_criteria = 'byrmse'
print(f'Model_name: {model_name}, Select_criteria: {select_criteria}')

Model_name: Mlp, Select_criteria: byrmse


Intra-dataset: diva-vqa_patch16_224_top196, slowfast+swint_base, epoch = 200

In [4]:
log_path = f'../log/best_model/'
feats_name = ['diva-vqa']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc', 'youtube_ugc_h264', 'lsvq_test', 'lsvq_test_1080p']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,diva-vqa,0.8686 (±0.0299),0.6886 (±0.0428),0.8924 (±0.0281),9.5136 (±1.0760)
1,konvid_1k,diva-vqa,0.8563 (±0.0160),0.6723 (±0.0190),0.8621 (±0.0152),0.3263 (±0.0119)
2,live_vqc,diva-vqa,0.8249 (±0.0283),0.6375 (±0.0316),0.8584 (±0.0262),8.7821 (±0.7984)
3,youtube_ugc,diva-vqa,0.8206 (±0.0222),0.6283 (±0.0211),0.8333 (±0.0189),0.3577 (±0.0145)
4,youtube_ugc_h264,diva-vqa,0.8229 (±0.0312),0.6288 (±0.0334),0.8325 (±0.0249),0.3585 (±0.0199)
5,lsvq_test,diva-vqa,0.8770 (±0.0013),0.6934 (±0.0016),0.8773 (±0.0011),5.0452 (±0.0213)
6,lsvq_test_1080p,diva-vqa,0.7886 (±0.0024),0.5982 (±0.0025),0.8320 (±0.0020),5.7556 (±0.0314)


Intra-dataset: diva-vqa_patch16_224_top196, slowfast+swint_large, epoch = 200

In [5]:
log_path = f'../log/best_model/'
feats_name = ['diva-vqa_large']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc', 'youtube_ugc_h264', 'lsvq_test', 'lsvq_test_1080p']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,diva-vqa_large,0.8712 (±0.0312),0.7040 (±0.0437),0.8960 (±0.0277),9.7853 (±1.0557)
1,konvid_1k,diva-vqa_large,0.8695 (±0.0144),0.6840 (±0.0168),0.8733 (±0.0134),0.3130 (±0.0109)
2,live_vqc,diva-vqa_large,0.8243 (±0.0253),0.6398 (±0.0281),0.8600 (±0.0245),8.7648 (±0.7409)
3,youtube_ugc,diva-vqa_large,0.8291 (±0.0250),0.6335 (±0.0237),0.8350 (±0.0212),0.3551 (±0.0179)
4,youtube_ugc_h264,diva-vqa_large,0.8233 (±0.0304),0.6318 (±0.0335),0.8321 (±0.0250),0.3494 (±0.0221)
5,lsvq_test,diva-vqa_large,0.8810 (±0.0007),0.6991 (±0.0010),0.8805 (±0.0008),4.9833 (±0.0157)
6,lsvq_test_1080p,diva-vqa_large,0.7893 (±0.0025),0.5983 (±0.0027),0.8303 (±0.0025),5.7817 (±0.0378)


Cross-dataset: diva-vqa_patch16_224_top196, train on LSVQ, epoch = 200

In [6]:
is_finetune = True
select_criteria = 'byrmse'
print(f'Model_name: diva-vqa(train on LSVQ), is_finetune:{is_finetune}, Select_criteria: {select_criteria}')
log_path = f'../log/reported_results/fine_tune/'
feats_name = ['diva-vqa', 'diva-vqa_large']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc', 'youtube_ugc_h264']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

Model_name: diva-vqa(train on LSVQ), is_finetune:True, Select_criteria: byrmse


,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,diva-vqa_finetune,0.899945,0.749653,0.921790,8.289659
1,konvid_1k,diva-vqa_finetune,0.891916,0.718300,0.900210,0.289666
2,live_vqc,diva-vqa_finetune,0.895068,0.728559,0.923710,7.539412
3,youtube_ugc,diva-vqa_finetune,0.866756,0.682247,0.873137,0.332065
4,youtube_ugc_h264,diva-vqa_finetune,0.857709,0.668184,0.873126,0.316847
5,cvd_2014,diva-vqa_large_finetune,0.910615,0.772433,0.917225,8.393226
6,konvid_1k,diva-vqa_large_finetune,0.904744,0.730514,0.906596,0.275602
7,live_vqc,diva-vqa_large_finetune,0.894813,0.728264,0.923809,7.534713
8,youtube_ugc,diva-vqa_large_finetune,0.865080,0.680189,0.871277,0.320954
9,youtube_ugc_h264,diva-vqa_large_finetune,0.881000,0.697612,0.877357,0.316830


In [7]:
is_finetune = False
select_criteria = 'byrmse'
print(f'Model_name: diva-vqa(train on LSVQ), is_finetune:{is_finetune}, Select_criteria: {select_criteria}')
log_path = f'../log/reported_results/fine_tune/'
feats_name = ['diva-vqa', 'diva-vqa_large']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc', 'youtube_ugc_h264']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

Model_name: diva-vqa(train on LSVQ), is_finetune:False, Select_criteria: byrmse


,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,diva-vqa_wo_finetune,0.840337,0.649438,0.848405,11.350764
1,konvid_1k,diva-vqa_wo_finetune,0.849164,0.660686,0.857332,0.329881
2,live_vqc,diva-vqa_wo_finetune,0.806975,0.614233,0.836093,9.357605
3,youtube_ugc,diva-vqa_wo_finetune,0.736699,0.542183,0.754135,0.427654
4,youtube_ugc_h264,diva-vqa_wo_finetune,0.733756,0.538658,0.750609,0.426608
5,cvd_2014,diva-vqa_large_wo_finetune,0.849544,0.660736,0.854300,11.145607
6,konvid_1k,diva-vqa_large_wo_finetune,0.862176,0.675475,0.866332,0.320079
7,live_vqc,diva-vqa_large_wo_finetune,0.823906,0.631515,0.848956,9.014339
8,youtube_ugc,diva-vqa_large_wo_finetune,0.748428,0.551100,0.763719,0.420381
9,youtube_ugc_h264,diva-vqa_large_wo_finetune,0.750185,0.552542,0.765328,0.415563


Ablation study: patch_size

pre-train model = 'swin_base_patch4_window7_224.ms_in22k_ft_in1k' & 'swin_base_patch4_window12_384.ms_in22k_ft_in1k'

resize=224, 384, patch_size=8,16,32, create plot:Plcc-flops-time

In [8]:
log_path = f'../features_ablation/log/'
feats_name = ['diva-vqa_patch8_224_top784', 'diva-vqa_patch16_224_top196', 'diva-vqa_patch32_224_top49', 'diva-vqa_base_384_patch8_384_top2304', 'diva-vqa_base_384_patch16_384_top576', 'diva-vqa_base_384_patch32_384_top144']
data_name = ['konvid_1k']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,diva-vqa_patch8_224_top784,0.8438 (±0.0144),0.6568 (±0.0166),0.8466 (±0.0152),0.3419 (±0.0125)
1,konvid_1k,diva-vqa_patch16_224_top196,0.8537 (±0.0156),0.6665 (±0.0185),0.8576 (±0.0151),0.3299 (±0.0117)
2,konvid_1k,diva-vqa_patch32_224_top49,0.8509 (±0.0174),0.6644 (±0.0203),0.8530 (±0.0172),0.3298 (±0.0134)
3,konvid_1k,diva-vqa_base_384_patch8_384_top2304,0.8489 (±0.0155),0.6633 (±0.0180),0.8555 (±0.0146),0.3341 (±0.0117)
4,konvid_1k,diva-vqa_base_384_patch16_384_top576,0.8470 (±0.0158),0.6581 (±0.0178),0.8479 (±0.0153),0.3365 (±0.0127)
5,konvid_1k,diva-vqa_base_384_patch32_384_top144,0.8455 (±0.0164),0.6578 (±0.0184),0.8528 (±0.0166),0.3343 (±0.0126)


Ablation study: pre-train model

patch_size = patch16

In [9]:
log_path = f'../features_ablation/log/'
feats_name = ['diva-vqa_tiny', 'diva-vqa_small', 'diva-vqa', 'diva-vqa_large', 'diva-vqa_base_384_patch16_384_top576', 'diva-vqa_large_384_patch16_384_top576']
data_name = ['konvid_1k']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,diva-vqa_tiny,0.8370 (±0.0173),0.6485 (±0.0189),0.8447 (±0.0154),0.3401 (±0.0113)
1,konvid_1k,diva-vqa_small,0.8462 (±0.0170),0.6655 (±0.0197),0.8577 (±0.0161),0.3326 (±0.0118)
2,konvid_1k,diva-vqa,0.8563 (±0.0160),0.6723 (±0.0190),0.8621 (±0.0152),0.3263 (±0.0119)
3,konvid_1k,diva-vqa_large,0.8695 (±0.0144),0.6840 (±0.0168),0.8733 (±0.0134),0.3130 (±0.0109)
4,konvid_1k,diva-vqa_base_384_patch16_384_top576,0.8470 (±0.0158),0.6581 (±0.0178),0.8479 (±0.0153),0.3365 (±0.0127)
5,konvid_1k,diva-vqa_large_384_patch16_384_top576,0.8638 (±0.0141),0.6752 (±0.0162),0.8670 (±0.0141),0.3185 (±0.0114)


Ablation study: sampling fps
diva-vqa_base = diva-vqa = diva-vqa_patch16_224_top196

In [10]:
log_path = f'../features_ablation/log/'
feats_name = ['diva-vqa', 'diva-vqa_sub_patch16_224_top196']
data_name = ['konvid_1k']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,diva-vqa,0.8563 (±0.0160),0.6723 (±0.0190),0.8621 (±0.0152),0.3263 (±0.0119)
1,konvid_1k,diva-vqa_sub_patch16_224_top196,0.8457 (±0.0129),0.6612 (±0.0166),0.8560 (±0.0131),0.3329 (±0.0108)


best: diva-vqa_patch16_224_top196, swin_large_patch4_window7_224.ms_in22k_ft_in1k, full sampling